# 🚀 **Satellite Image Classification with PyTorch**  

## By Kao Panboonyuen

### This Colab notebook will guide you through:

* ✅ Preparing and loading the dataset
* ✅ Exploring images and labels
* ✅ Performing exploratory data analysis (EDA)
* ✅ Splitting the dataset into Train/Val/Test (with fixed seed)
* ✅ Training 3 models:
     - 🏗️ Simple CNN (from scratch)
     - 🔥 ResNet18 (Pretrained)
     - 🏆 DenseNet (Pretrained)
* ✅ Saving & loading model weights
* ✅ Evaluating performance with accuracy, confusion matrix, precision, recall, F1-score
* ✅ Visualizing correct & incorrect predictions for error analysis


### 📌 **Step 1: Install & Import Required Libraries**

In [ ]:
!pip install torch==2.0.0+cu117 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score, f1_score
import os
import zipfile
import requests
import random
import torch.nn.functional as F

import warnings
warnings.filterwarnings("ignore")

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

In [ ]:
# Download dataset
!wget https://github.com/kaopanboonyuen/OCSB-AI/raw/main/dataset/satellite-dataset.zip -O satellite-dataset.zip
!unzip satellite-dataset.zip -d dataset

### 📌 **Step 2: Load & Prepare Dataset**

In [ ]:
# Define dataset path
dataset_path = 'Write your dataset path here'

# Define image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

In [ ]:
# Load dataset
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
classes = dataset.classes  # Get class names
print("Dataset classes:", classes)

In [ ]:
# Split dataset (70% train, 15% validation, 15% test)
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

In [ ]:
# DataLoader (Batch Size = 32)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

### 📌 **Step 3: Preview Dataset**

In [ ]:
# Visualize a few images
import matplotlib.pyplot as plt
import random

def show_images(dataset, num=6):
    fig, axes = plt.subplots(1, num, figsize=(15, 5))
    indices = random.sample(range(len(dataset)), num)  # Select random indices
    for i, idx in enumerate(indices):
        img, label = dataset[idx]
        axes[i].imshow(img.permute(1, 2, 0))
        axes[i].set_title(classes[label])
        axes[i].axis('off')
    plt.show()

show_images(train_dataset)

In [ ]:
show_images(train_dataset)

In [ ]:
show_images(train_dataset)

In [ ]:
show_images(train_dataset)

### 📌 **Step 4: Define Models**

#### **Model 1: CNN from Scratch**

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(64 * 56 * 56, 128)
        self.fc2 = nn.Linear(128, len(classes))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

#### **Model 2: Pretrained ResNet18**

In [ ]:
# Write your resnet18 model here

for param in model_resnet.parameters():
    param.requires_grad = False
model_resnet.fc = nn.Linear(model_resnet.fc.in_features, len(classes))

#### **Model 3: Pretrained DenseNet**

In [ ]:
# Write your densenet model here

for param in model_densenet.parameters():
    param.requires_grad = False
model_densenet.classifier = nn.Linear(model_densenet.classifier.in_features, len(classes))

### 📌 **Step 5: Train Function**

In [ ]:
def train_model(model, train_loader, val_loader, epochs=10, learning_rate=0.001):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    train_losses, val_losses, train_accs, val_accs = [], [], [], []

    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_losses.append(running_loss / len(train_loader))
        train_accs.append(correct / total)

        print(f"Epoch {epoch+1}: Train Loss = {train_losses[-1]:.4f}, Train Acc = {train_accs[-1]:.4f}")

    print("Training Complete!")
    return model, train_losses, train_accs

### 📌 **Step 6: Train Models**

In [ ]:
# Train Simple CNN

# Write your trainer code here

In [ ]:
# Train ResNet

# Write your trainer code here

In [ ]:
# Train DenseNet

# Write your trainer code here

### 📌 **Step 7: Evaluate Performance**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

# Function to plot confusion matrix
def plot_confusion_matrix(true_labels, predictions, classes):
    cm = confusion_matrix(true_labels, predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=classes, yticklabels=classes)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

def evaluate_model(model, test_loader, classes):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    predictions, true_labels = [], []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Calculate performance metrics
    acc = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='weighted')
    recall = recall_score(true_labels, predictions, average='weighted')
    f1 = f1_score(true_labels, predictions, average='weighted')

    print(classification_report(true_labels, predictions, target_names=classes, digits=4))

    # Plot confusion matrix
    plot_confusion_matrix(true_labels, predictions, classes)

    return acc, precision, recall, f1

In [ ]:
evaluate_model(model1, test_loader, classes)
evaluate_model(model_resnet, test_loader, classes)
evaluate_model(model_densenet, test_loader, classes)

📌 Step 9: Save, Load, and Inference Model Weights

In [ ]:
# Save the trained model (DenseNet)
# Write your code here
print("Model DenseNet saved to DenseNet_model.pth")

In [ ]:
import torch
import torchvision.models as models
import torch.nn as nn

# Load the pre-trained DenseNet model structure
model_densenet = models.densenet121(weights=models.DenseNet121_Weights.IMAGENET1K_V1)

# Modify the final classifier layer to match your number of classes
model_densenet.classifier = nn.Linear(model_densenet.classifier.in_features, len(classes))

# Load the saved model weights
model_densenet.load_state_dict(torch.load('DenseNet_model.pth'))
print("Model DenseNet loaded from DenseNet_model.pth")

# Set the model to evaluation mode (important for inference)
model_densenet.eval()

# Now you can perform inference

### 📌 Step 10: Inference Model (Deployment)

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# Function to perform inference
def predict_image(image_path, model, classes, transform):
    # Load image and convert to RGB (in case it has 4 channels like RGBA)
    img = Image.open(image_path).convert('RGB')

    # Apply the transformations (resize and tensor conversion)
    img = transform(img).unsqueeze(0)  # Add batch dimension

    # Send the image to the same device as the model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    img = img.to(device)

    # Perform inference
    with torch.no_grad():
        output = model(img)
        _, predicted_class = torch.max(output, 1)

    # Display the predicted class
    print(f"Predicted class: {classes[predicted_class]}")

    # Show the image
    img_show = img.squeeze().cpu().numpy().transpose((1, 2, 0))  # Convert tensor to numpy for plotting
    plt.imshow(img_show)
    plt.title(f"Predicted: {classes[predicted_class]}")
    plt.axis('off')
    plt.show()

# Example image path (replace with your image file)
image_path = '/content/dataset/satellite-dataset/cloudy/train_10021.jpg'

# Apply the same transformations used during training
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Run inference
predict_image(image_path, model_densenet, classes, transform)